In [82]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [83]:
df = pd.read_csv("C:\\Users\\Lenovo\\Desktop\\Project\\Customer attrition using logistic regression on bank dataset\BankChurners.csv")
df

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.999910
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.999940
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.999980
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.999870
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.999980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,772366833,Existing Customer,50,M,2,Graduate,Single,$40K - $60K,Blue,40,...,4003.0,1851,2152.0,0.703,15476,117,0.857,0.462,0.000191,0.999810
10123,710638233,Attrited Customer,41,M,2,Unknown,Divorced,$40K - $60K,Blue,25,...,4277.0,2186,2091.0,0.804,8764,69,0.683,0.511,0.995270,0.004729
10124,716506083,Attrited Customer,44,F,1,High School,Married,Less than $40K,Blue,36,...,5409.0,0,5409.0,0.819,10291,60,0.818,0.000,0.997880,0.002118
10125,717406983,Attrited Customer,30,M,2,Graduate,Unknown,$40K - $60K,Blue,36,...,5281.0,0,5281.0,0.535,8395,62,0.722,0.000,0.996710,0.003294


In [84]:
# Checking columns list and missing values
df.isnull().sum()

CLIENTNUM                                                                                                                             0
Attrition_Flag                                                                                                                        0
Customer_Age                                                                                                                          0
Gender                                                                                                                                0
Dependent_count                                                                                                                       0
Education_Level                                                                                                                       0
Marital_Status                                                                                                                        0
Income_Category                                 

In [85]:
def binary_encode(df, column, positive_value):
    df = df.copy()
    df[column] = df[column].apply(lambda x: 1 if x == positive_value else 0)
    return df

def ordinal_encode(df, column, ordering):
    df = df.copy()
    df[column] = df[column].apply(lambda x: ordering.index(x))
    return df

def onehot_encode(df, column, prefix):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=prefix)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [86]:
def preprocess_inputs(df):
    df = df.copy()
    
    #Drop last two columns
    df= df.drop(df.columns[-2:], axis=1)
    
    # Drop CLIENTNUM columns
    df= df.drop('CLIENTNUM', axis=1)
    
    # Encode unknown values as np.NaN
    df = df.replace('Unknown', np.NaN)
    
     # filling ordinal missing values with modes[Education_level and Income_Category columns]
    df['Education_Level'] = df['Education_Level'].fillna('Graduate')
    df['Income_Category'] = df['Income_Category'].fillna('Less than $40K')
    
    #encode binary columns
    df = binary_encode(df, 'Attrition_Flag', positive_value='Attrited Customer')
    df = binary_encode(df, 'Gender', positive_value='M')
    
    #encode ordinal columns
    education_ordering=[
         'Uneducated',
         'High School',
         'College',
         'Graduate',
         'Post-Graduate',
         'Doctorate',
    ]
    income_ordering = [
        'Less than $40K',
        '$40K - $60K',
        '$60K - $80K', 
        '$80K - $120K',
        '$120K +',
    ]
    df = ordinal_encode(df, 'Education_Level' , ordering=education_ordering)
    df = ordinal_encode(df, 'Income_Category' , ordering=income_ordering)
    
    #encode nominal columns
    df = onehot_encode(df, 'Marital_Status', prefix='MS')
    df = onehot_encode(df, 'Card_Category', prefix='CC')
  
    #Split df into X and Y
    y = df['Attrition_Flag'].copy()
    x = df.drop('Attrition_Flag', axis=1).copy()
    
    #Scale x with standard Scaler
    scaler = StandardScaler()
    x = pd.DataFrame(scaler.fit_transform(x), columns=x.columns)
    return x, y
     

    

In [87]:
X, y = preprocess_inputs(df)

In [88]:
y

0        0
1        0
2        0
3        0
4        0
        ..
10122    0
10123    1
10124    1
10125    1
10126    1
Name: Attrition_Flag, Length: 10127, dtype: int64

In [89]:
X

,Customer_Age,Gender,Dependent_count,Education_Level,Income_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,...,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,MS_Divorced,MS_Married,MS_Single,CC_Blue,CC_Gold,CC_Platinum,CC_Silver
0,-0.165406,1.059956,0.503368,-0.893680,0.597300,0.384621,0.763943,-1.327136,0.492404,0.446622,...,-0.973895,3.834003,-0.775882,-0.282405,1.077338,-0.798507,0.270611,-0.107644,-0.044484,-0.240794
1,0.333570,-0.943436,2.043199,0.593388,-0.887628,1.010715,1.407306,-1.327136,-0.411616,-0.041367,...,-1.357340,12.608573,-0.616276,-0.282405,-0.928214,1.252337,0.270611,-0.107644,-0.044484,-0.240794
2,0.583058,1.059956,0.503368,0.593388,1.339764,0.008965,0.120579,-1.327136,-2.219655,-0.573698,...,-1.911206,6.807864,-0.997155,-0.282405,1.077338,-0.798507,0.270611,-0.107644,-0.044484,-0.240794
3,-0.789126,-0.943436,1.273283,-0.893680,-0.887628,-0.241473,-0.522785,1.641478,-1.315636,-0.585251,...,-1.911206,6.807864,1.759686,-0.282405,-0.928214,-0.798507,0.270611,-0.107644,-0.044484,-0.240794
4,-0.789126,1.059956,0.503368,-1.637214,0.597300,-1.869317,0.763943,-1.327136,-2.219655,-0.430877,...,-1.570365,7.509325,-0.997155,-0.282405,1.077338,-0.798507,0.270611,-0.107644,-0.044484,-0.240794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,0.458314,1.059956,-0.266547,0.593388,-0.145164,0.509840,-0.522785,-0.337598,0.492404,-0.509330,...,2.221481,0.608119,0.678714,-0.282405,-0.928214,1.252337,0.270611,-0.107644,-0.044484,-0.240794
10123,-0.664382,1.059956,-0.266547,0.593388,-0.145164,-1.368442,0.120579,-0.337598,0.492404,-0.479181,...,0.176440,-0.122745,0.856458,3.541013,-0.928214,-0.798507,0.270611,-0.107644,-0.044484,-0.240794
10124,-0.290150,-0.943436,-1.036462,-0.893680,-0.887628,0.008965,0.763943,0.651940,1.396424,-0.354626,...,-0.207005,0.444305,-0.997155,-0.282405,1.077338,-0.798507,0.270611,-0.107644,-0.044484,-0.240794
10125,-2.036565,1.059956,-0.266547,0.593388,-0.145164,0.008965,0.120579,0.651940,0.492404,-0.368710,...,-0.121795,0.041070,-0.997155,-0.282405,-0.928214,-0.798507,0.270611,-0.107644,-0.044484,-0.240794


In [90]:
y

0        0
1        0
2        0
3        0
4        0
        ..
10122    0
10123    1
10124    1
10125    1
10126    1
Name: Attrition_Flag, Length: 10127, dtype: int64

In [63]:
data['Attrition_Flag'].unique()

array(['Existing Customer', 'Attrited Customer'], dtype=object)

In [64]:
{column: len(data[column].unique()) for column in data.select_dtypes('object').columns}

{'Attrition_Flag': 2,
 'Gender': 2,
 'Education_Level': 7,
 'Marital_Status': 4,
 'Income_Category': 6,
 'Card_Category': 4}

In [65]:
{column: list(data[column].unique()) for column in data.select_dtypes('object').columns}

{'Attrition_Flag': ['Existing Customer', 'Attrited Customer'],
 'Gender': ['M', 'F'],
 'Education_Level': ['High School',
  'Graduate',
  'Uneducated',
  'Unknown',
  'College',
  'Post-Graduate',
  'Doctorate'],
 'Marital_Status': ['Married', 'Single', 'Unknown', 'Divorced'],
 'Income_Category': ['$60K - $80K',
  'Less than $40K',
  '$80K - $120K',
  '$40K - $60K',
  '$120K +',
  'Unknown'],
 'Card_Category': ['Blue', 'Gold', 'Silver', 'Platinum']}

In [66]:
X.isna().sum()

Customer_Age                0
Gender                      0
Dependent_count             0
Education_Level             0
Income_Category             0
Months_on_book              0
Total_Relationship_Count    0
Months_Inactive_12_mon      0
Contacts_Count_12_mon       0
Credit_Limit                0
Total_Revolving_Bal         0
Avg_Open_To_Buy             0
Total_Amt_Chng_Q4_Q1        0
Total_Trans_Amt             0
Total_Trans_Ct              0
Total_Ct_Chng_Q4_Q1         0
Avg_Utilization_Ratio       0
MS_Divorced                 0
MS_Married                  0
MS_Single                   0
CC_Blue                     0
CC_Gold                     0
CC_Platinum                 0
CC_Silver                   0
dtype: int64

In [67]:
X.mean

<bound method NDFrame._add_numeric_operations.<locals>.mean of        Customer_Age    Gender  Dependent_count  Education_Level  \
0         -0.165406  1.059956         0.503368        -0.893680   
1          0.333570 -0.943436         2.043199         0.593388   
2          0.583058  1.059956         0.503368         0.593388   
3         -0.789126 -0.943436         1.273283        -0.893680   
4         -0.789126  1.059956         0.503368        -1.637214   
...             ...       ...              ...              ...   
10122      0.458314  1.059956        -0.266547         0.593388   
10123     -0.664382  1.059956        -0.266547         0.593388   
10124     -0.290150 -0.943436        -1.036462        -0.893680   
10125     -2.036565  1.059956        -0.266547         0.593388   
10126     -0.414894 -0.943436        -0.266547         0.593388   

       Income_Category  Months_on_book  Total_Relationship_Count  \
0             0.597300        0.384621                  0.763943

In [68]:
data['Education_Level'].mode()

0    Graduate
Name: Education_Level, dtype: object

In [69]:
def binary_encode(df ,column, positive_value):
    df = df.copy()
    df[column] = df[column].apply(lambda x: 1 if x == positive_value else 0)
    return df

def ordinal_encode(df, column, ordering):
    df = df.copy()
    df[column] = df[column].apply(lambda x: ordering.index(x))
    return df

def onehot_encode(df, column, prefix):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=prefix)
    df= pd.concat([df, dummies], axis=1)
    df = df.drop(column , axis=1)
    return df

In [70]:
pd.get_dummies(data['Card_Category'], prefix='CARD')

,CARD_Blue,CARD_Gold,CARD_Platinum,CARD_Silver
0,1,0,0,0
1,1,0,0,0
2,1,0,0,0
3,1,0,0,0
4,1,0,0,0
...,...,...,...,...
10122,1,0,0,0
10123,1,0,0,0
10124,1,0,0,0
10125,1,0,0,0


Training

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.7, random_state=123)

In [74]:
models = [
    LogisticRegression(),
    DecisionTreeClassifier(),
    RandomForestClassifier()
]

for model in models:
    model.fit(X_train, y_train)

In [77]:
model_names = [
    "   Logistic Regression",
    "         Decision Tree",
    "         Random Forest"
]

for model, name in zip(models, model_names):
    print(name + ": {:.4f}%".format(model.score(X_test, y_test) * 100))

   Logistic Regression: 90.4574%
         Decision Tree: 93.8138%
         Random Forest: 95.6894%
